In [1]:
!pip install python_speech_features

from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random
import operator

import math
import numpy as np

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=27f8a0741ce9fe7de6cf21b488ed026e0b3941320c8bee968255146f21d4d50e
  Stored in directory: /root/.cache/pip/wheels/37/01/19/e6c69a32684ab7b2e3ea4985a571d810cf055c72600e7f9f17
Successfully built python_speech_features


In [2]:
# def getNeighbors(trainingSet, instance, k):
#     distances = []
#     for x in range (len(trainingSet)):
#         dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
#         distances.append((trainingSet[x][2], dist))
#     distances.sort(key=operator.itemgetter(1))
#     neighbors = []
#     for x in range(k):
#         neighbors.append(distances[x][0])
#     return neighbors

In [3]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [4]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sfwsajal","key":"442bd984473decb9792f939ea42ad598"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification


Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
100% 1.21G/1.21G [00:13<00:00, 161MB/s]
100% 1.21G/1.21G [00:13<00:00, 97.4MB/s]


In [10]:
import zipfile
import os

# Extracting the zip file
with zipfile.ZipFile("gtzan-dataset-music-genre-classification.zip", 'r') as zip_ref:
    zip_ref.extractall("gtzan_data")

# Verify the contents
os.listdir("gtzan_data")


['Data']

In [13]:
directory = "/content/gtzan_data/Data/genres_original/"
f = open("my.dat", 'wb')
i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break  # Only process first 10 genres

    genre_path = os.path.join(directory, folder)

    for file in os.listdir(genre_path):
        try:
            if not file.endswith(".wav"):
                continue  # Skip non-audio files

            file_path = os.path.join(genre_path, file)

            #  Loading WAV audio using scipy
            rate, sig = wav.read(file_path)

            # Extracting the MFCC features
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.transpose(mfcc_feat))
            mean_vector = mfcc_feat.mean(0)

            # Storing the (mean, covariance, label) as feature
            feature = (mean_vector, covariance, i)
            pickle.dump(feature, f)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

f.close()


Error processing /content/gtzan_data/Data/genres_original/jazz/jazz.00054.wav: File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF', 'RIFX', and 'RF64' supported.


In [22]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

    for x in range(len(dataset)):
        if random.random() <split :
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.8, trainingSet, testSet)

In [23]:
import numpy as np

def distance(instance1, instance2, k):
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]

    # Bhattacharyya distance formula
    dist = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    dist += np.dot(np.dot((mm2 - mm1).T, np.linalg.inv(cm2)), (mm2 - mm1))
    dist += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    dist -= k
    return dist


In [24]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [29]:
leng = len(testSet)
k_values = [1, 3, 5, 7, 9, 11]
for k in k_values:
    predictions = []
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        predictions.append(nearestClass(neighbors))

    accuracy = getAccuracy(testSet, predictions)
    print(f"Accuracy for k = {k}: {accuracy:.4f}")

# predictions = []
# for x in range (leng):
#     predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5)))

# accuracy1 = getAccuracy(testSet , predictions)
# print(accuracy1)

Accuracy for k = 1: 0.7005
Accuracy for k = 3: 0.7005
Accuracy for k = 5: 0.7259
Accuracy for k = 7: 0.7107
Accuracy for k = 9: 0.6954
Accuracy for k = 11: 0.6802


In [ ]:
# so above we can see we acheived the highest accuracy as 72.59% for k=5.

In [26]:
from google.colab import files
files.download("my.dat")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>